全卷积网络-Full Convolutional Net(FCN)
===

# 1.概述

目前在图像分割领域比较成功的算法，有很大一部分都来自于同一个先驱：Long等人提出的Fully Convolutional Network（FCN）。

FCN将分类网络转换成用于分割任务的网络结构，并证明了在分割问题上，可以实现端到端的网络训练。基于此，FCN成为了深度学习解决分割问题的奠基石。

目标识别网络（分类网络）尽管表面上来看可以接受任意尺寸的图片作为输入，但是由于网络结构最后全连接层的存在，使其丢失了输入的空间信息，因此，这些网络并没有办法直接用于解决诸如分割等稠密估计的问题。

考虑到这一点，FCN用卷积层和池化层替代了分类网络中的全连接层，从而使得网络结构可以适应像素级的稠密估计任务。这种全卷积网络结构不仅能够支持稠密估计，而且能够实现端到端的训练。

# 2.介绍
![images](...%0AFABRRRQAUUUUAFFFFABRRRQB/9k=)


![images](...%0AooooAKKKKACiiigAooooA//Z)

- 输入：整幅图像。
- 输出：空间尺寸与输入图像相同，通道数等于全部类别个数。
- 真值：通道数为1（或2）的分割图像。

## 2.1.全连接层转换成卷积层
将全连接层替换成卷积层的优势之一，是可以使网络用于稠密估计任务，并实现端到端训练。比如下图中，将全连接层替换成卷积层后，可以使得网络输出一个热度图（heatmap），而非单个类别标签。而这种做法的另外一个优势就是，通过接收整幅图像作为输入，而非以图块的形式处理图片，网络在处理速度上也有了明显提升。

## 2.2.连接不同尺度下的层
分类网络通常会通过设置步长的方式逐渐减小每层的空间尺寸，这种方式可以同时实现计算量的缩小和信息的浓缩。尽管这种操作对于分类任务是很有效的，但是对于分割这样需要稠密估计的任务而言，这种浓缩就未必是好事了。

比如下面这张图就是全局步长32下的分割效果。虽然实现了分割，但是结果很粗糙，看不出来目标的细节。

![images](data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsL%0ADBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/%0A2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIy%0AMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCAD0AKUDASIAAhEBAxEB/8QA%0AHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUF%0ABAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkK%0AFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1%0Adnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXG%0Ax8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEB%0AAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAEC%0AAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRom%0AJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOE%0AhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU%0A1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD0zxVrmo6Z%0AqcUFnceVG0IcjYp53MO49qwW8Wa5/wA/3/kJP8Ku+OP+Q3D/ANe6/wDoTVzJ%0ArFyfMelQpwcE2jkfHPxV8baNrwtbDW/Kh8lW2/ZYW5JPqhPpXM/8Ls+If/Qw%0A/wDklb//ABuuu8WeFIPEUAkQiG9jGElxkEf3Wxzjrj0rxa4ge1uZLeUYkiYo%0A49CDgitIu5x16fJLTY7r/hdnxD/6GH/yTt//AI3R/wALs+If/Qw/+Sdv/wDG%0A64CimZHf/wDC7PiH/wBDD/5J2/8A8bpf+F2fEP8A6GH/AMkrf/43Xn9FAz6N%0A8GePvE2reHIby91PzZ2dwW8iNcgHA4CgVv8A/CV63/z/AH/kJP8ACuV0LTzp%0AOg2NkVCtHEA4H948t+pNaIOaycnfQ9OlSjyrmRs/8JXrf/P8f+/Sf4V5T4n+%0ALnjnTvEl9aWmueXBG4Cp9kgOBgdyma72vOPiL4ZaVf7atEJZQFuVHJIxw/6c%0A/gfU04z1szPEUVy3iil/wuz4hf8AQw/+SVv/APG6X/hdfxC/6GH/AMkrf/43%0AXAYpc1oead7/AMLs+If/AEMP/klb/wDxuj/hdnxD/wChh/8AJK3/APjdcBRQ%0AB3//AAuz4h/9DD/5JW//AMbo/wCF2fEP/oYf/JK3/wDjdcBRQB3/APwuz4h/%0A9DD/AOSVv/8AG6P+F2fEP/oYf/JK3/8AjdcBRQB9mfC7WtR8R/DrS9V1W5Nx%0Aez+b5kuxU3bZXUcKABwAOlFUfgrx8I9D/wC2/wD6PkooAi8c/wDIbh/691/9%0ACauYFdP45/5DcP8A17r/AOhNXM1hL4j16H8JCYrxf4had9h8UyyKCEukEwyc%0A89D+oJ/Gvaa8v+KsSi/02bq7RMpPqARj+Zq4PUjFRvTued0ZoorY8wWtzwhp%0Ay6p4nsrdx+7EnmP9F+bH44x+NYdeg/Cyy36jfXeAfKiEYyOMsc/+y/rUydka%0A0oqU0meo0UUVzt3PXFpjLuUjjBGDkZp9FIT1PNPE3w6kaV7vRQhDEl7YnGD/%0AALPt14rzy4t5rWd4Z4mjlQ4ZGGCDX0dXM+KvB9v4hh82JlhvUHySkcMP7re3%0Av2961jLuclbCp6xPEqKvanpN7o90be9geJ+2Rww9Qe9Ua0PPaadmFFFaWh6V%0ALrWr29hFnMjfMw/hUck/lQCTbsi9p/gzXNSto7mCzxbyDKyPIqgj6E5/St61%0A+FmoSc3d9BAvcIpdh+HH869Qgijt7eK3hTbFEgRF9FAwBU1Z856McLG2p3vw%0A20pNF8A6Zp8crSLD5vzsOTmV2P8AOitHwj/yLNp9X/8AQzRWq1RwTilJpHK+%0AOf8AkNQ/9e6/+hNXM103jn/kNQ/9e6/+hNXM1zy+I9Sh/CQtef8AxVjzpmny%0A5+7My4x1yM/0r0CuC+Kn/IEsv+vj/wBlNVDcMR/CZ5RRRRW55Ite0fD2wOn+%0AF45GXEl0xlPsOg/QZ/GvKNC0uTWdYt7GPI8x/mYfwqOSfyBr32KNIYo4Y12x%0AxqEUegHSsqj0sduEhrzMkooorE9AKKKKBBR3oooGVNR0yz1a0a1vYFlibseo%0APqD2PNeT+KPAt3oaPdW7fabEdXGAyD/aH9R+lex0kkaSxskihlYYIIyCKuM7%0AGNWjGprsz5yt4JbmdIYUZ5HYKqqMkk17P4N8Kp4fs3luArahMMSMP4Bwdg/E%0Acn+dLo3gjTtG1ea/iZpN3EMbgERZ689T7Hjj1rphVOSZjQw/L7zCiiioOo9P%0A8If8ixafV/8A0M0UeEP+RYtPq/8A6GaK3Wx49T42ct46/wCQ1D/17r/6E1cz%0AXTeOv+Q1D/17r/6E1czWEviPTofwkLXBfFT/AJAll/18f+ymu9rgPirIBpVh%0AF/E0xYfgP/riqhuGI/hM8rooorc8k9T+GejiGwm1VwfMnPlxnHRB1x9T1+ld%0A/WJ4PtvsnhLToz1MXmHj+8d39a3K55u7PXoxUYJISiiioNgooooAKKKKACii%0AigAooooEJRRRTA9Q8If8ixafV/8A0M0UeEP+RYtPq/8A6GaK6I7HjVPjZy3j%0Ar/kNQ/8AXuv/AKE1czXTeOf+Q3D/ANe6/wDoTVzNYS+I9PD/AMJC15t8VZ0L%0AaZADl18xz9DtA/ka9Av7+2020e6u5liiQcsx6nrgep46V4b4m1pte1y4vTkR%0Asdsan+FR0/x/E1cFrcjFVIqHL1MinIpeRVHUnFNqa1UveQgAljIoAA6nNas8%0A1H0NbQC3tooV+5GgQfgMVNRRXO9z2kNooopDFooooGFFFFABRRRQAUUUUAJR%0ARTJpUgheVzhEUsx9AKEI9Q8FyxzeFrVonDgNIpI6BhIwI/AgiisX4P3D3nwx%0A0u6k/wBZNJcyN9TcSk0V0LY8WbvJsp+Of+Q3D/17r/6E1cPrmtWug6Y95dNn%0AHEcY6yN2Aqr8ePE2r6J45srawu/KifTUkK+WrfMZZR3B7AV4vqesX+s3In1C%0A4aaQDaM4AA9gOBU8mtzqhiVGnyrcs65r17r13511IQik+XED8qD2/wAetZNJ%0AS1SVjklJyd2LVnTJUg1azmkOESdGY+wYVVyKM4pgtz6SorJ8N6iNU8O2V3kF%0A2jCvgY+ZeDx9c1rVzyVmezFpq6G0UuKTFIoWiiigYUUUUAFFFFABRRRQAVie%0AL53t/CepOnUxbPwYhT+hrbrD8YxCXwjqQJI2xbhj2IP9KFuiKnws9I+Cf/JI%0AtD/7eP8A0fJRR8Ev+SRaH/28f+j5KK6TxTyD9o3/AJKFYf8AYKj/APRs1eQV%0A6/8AtG/8lCsP+wVH/wCjZq8gpjCirdhp15qUpis7aWd+MhFzj3J7D61BPBJb%0AXEkEyFJY2KOp6gg4IoDUjooooA7DwX4x/sBms7xXewkbedgyUbAGQD2x/n19%0AK0/xVomqbVtb+MyNwI3+Rs+mD1rwWjNRKF9TenXlDTofSYOaK8I0zxZrOkYF%0AtduY+P3cnzr+R6fhXWaZ401PWYJoJmjhZcE+Su0sCT15+lYVf3cXI0q46NOD%0Am1sdvqGuW9kTGAZZR1VTwPqay28VyA4itVC99z55/IVgHmm4rzJYibeh81Vz%0AnEzb5XZHTab4ilur+OC4SJI3yoKAj5u3euirzgEqwZThlOQfQ139pN9ptYp+%0AodQfx710Yarze6z18mxs6vNCo7ssUUDpRXYe+FFFFABWV4miM3hfVEUgH7M5%0A59hn+latU9WjabRb+JfvSW0ijPqVIpdSZq8WjuPgl/ySLQ/+3j/0fJRS/BP/%0AAJJFof8A28f+j5KK6TxDx/8AaN/5KFYf9gqP/wBGzV5bpFl/aGrWtqW2rJIq%0As2cYXPJyfQZr2T486RJqnjyzmiljVU02NCGPOfNlP9a4fSdKjsIyx+adurHt%0AXPVxUKa8zlr4ynSXdnbLqOl6LYiy0mCMhegX7u71J7n35rz3xZaG5Z9TCjzW%0Af97tGAc9D/SugFV76Ez2U0f95D/LP+FebTxc/aJt6HnLMqtSrHm0R51RQRg0%0AV7Z7gUUUUAFdD4WjJuppeyqB+J6fyrnu9dtoNr9m01Cw+eQlzXLjJqNJrucW%0AOqclF+ZqUUUV4Z86LXT+GbnfbS2/eM7157H/AOv/ADrmKvaPOLbUoWJwHbyz%0A/wAC4H64rahPlmmd2XV/Y4iMvkdzRRRXrn3oUUUUAFZuvX8Wm6Hd3UvIWMgL%0AnlmIwB+ZrSrzP4ma4Hmh0eI5CYlnwe5HC/gOfxFVFGVafJFs95+Cf/JItD/7%0AeP8A0fJRSfBP/kkWh/8Abx/6PkorU8c4r4wf8jhb/wDXkn/ob1wHSvQPjB/y%0AOFv/ANeSf+hvXn9eFif4sj5nF/xmOooorBbnNHc84uBsuHTsrED86iq3qoxq%0Alz/11b+dU819JHVJn1sHeKFoop8UTTSrGgJZjgAVW2pTdjV0PS/t8zPKSsK8%0AEjqT6V2IQIoVeFHCj0FRWVqtnaRwqOVUbj6t3NWK8PE1vaz02R85jMQ6s/JD%0AaKKK5jkClBNJRQC0O90+6S9sYZ16svzfUcGrdc34ZvEHmWjNgk74wenTkfpn%0A866SvXo1OeCZ99gMQq9BS6hRRXl/jfxs1wz6VpcrLbg4mmU4LkH7o9vfv9Ou%0AyVzpq1FTV2dF4h8e6dpavBYst7dg4wp+RfqR1+gryO7u5r67lubhy8srFmY+%0ApqHNGa1UbHm1Ksqjuz68+Cf/ACSLQ/8At4/9HyUUfBP/AJJFof8A28f+j5KK%0AowOM+L//ACN9v/15J/6G9efV6D8X/wDkb7f/AK8k/wDQ3rz6vBxP8aR8zi/4%0AzHUUUVznL1OB1n/kLXP++aoVf1n/AJC9z/vmqFfSUvgR9ZS+BegtbPhu387U%0At56RLn8elYtdd4Vg2Wk0+P8AWMB+X/66zxM+Sm2ZYupyUWzeooorwT5kKKKK%0AAEooooGPjdo5UkUkFSCMexr0RGV40dTkMoYfSvOK7nR5vtGlW7ZztTZ9McV1%0A4V6tH0OQVWqkqb2Kviu9fT/DN/cIxV/KKKwzlSxCg/rXgzHJ5r174mzGPw3D%0AErgGW5UFc8kAE/zxXkBr1qa0PWxcm5i0UUVZzn1/8E/+SRaH/wBvH/o+Sij4%0AJ/8AJItD/wC3j/0fJRQI4v4v/wDI32//AF5J/wChvXn9egfF/wD5G+3/AOvJ%0AP/Q3rz+vBxP8aR8zi/4zFoopsjbI2bGcAmsErs5oq7scDqcgm1O4kXoXOKpU%0A5iS5PqabX0kVZJH1sVZJBXbeHGzpCD0YiuJFdl4ZOdMYekh/kK5ccr0jjzFf%0AuTZopcUmK8Y+fCiiigAooooAK7Pw7/yCF/66NXGV2Phr/kED/ro1dWE+M9rI%0Av95+Ry/xV/5Blh/11b+Qryw16b8V3YRaUmflJlYj1I2Y/ma8yxXsQ2PcxP8A%0AEYUUUVZgfX3wS/5JFof/AG8f+j5KKPgl/wAki0P/ALeP/R8lFIRxnxf/AORv%0At/8AryT/ANDevPq9B+L/APyN9v8A9eSf+hvXn1eDif40j5nF/wAZjqo6xcfZ%0AtLmkHXbtH48Vern/ABTcBbaKAHljuI9u386WGhz1UicJDnrRRyR60UUV9AfU%0AAK7Dwv8A8g1/+un9BXHiuw8L/wDIOk/66f0FcmN/hM4sw/gM3KKKK8Q+cCii%0AimMSiiigYVt6BqS2c5glYLFKfvE8K3/16xKbN/q2/wB0/wAqqEnF3R0YWvOj%0AVUoGL4/8QQa3q0SWjb7e2UqHB4ZifmI9uAM98VyVK3LHNJX0KR9PKTk7sbRR%0ARVCPr/4Jf8ki0P8A7eP/AEfJRR8Ev+SRaH/28f8Ao+SikBxnxf8A+Rvt/wDr%0AyT/0N68/rpfj3q8+m+OrKGJI2DabG5LDnPmyj+leR3Gu39wcebsXsE4xXmVc%0AHOpUck9GeRWy+dSo5X0Oynu7e2z508aY6gnn8q4nWL4318z5+RflT6VRZmYk%0AsxJPcmm11UMNGlruzrwuDjQd73YlFFFdJ2C11vhds2kyY+6+c/Uf/Wrkq6jw%0Ao3y3K47qf51zYxXpM48er0GdHRRRXhnzYUUUUxiUUUUDFo6gjsaKKENbnnM6%0AbJ5EPVWIqKtDWAE1a4UEH5+ceveqNfRRd0mfVwd4pjaKKKss+vvgl/ySLQ/+%0A3j/0fJRR8Ev+SRaH/wBvH/o+SikI8g/aN/5KFp//AGCo/wD0bLXkFev/ALR3%0A/JQtP/7BUf8A6NlryCmMSiiigQUUUUALXReFGPn3C54Kg/lXO1ueGJFTUXDH%0ABZCB9etYYlXpM58XHmoyR19FFFeAfLhRRRTGJRRRSGLVe/uhZ2UsxONq8fXo%0AP51YrmfE9992zU8gkyD+X9a3w9P2lRI6cLS9pVSOddjI7OxyWOSaSm5p2a90%0A+kGGig0Uxn1/8Ev+SRaH/wBvH/o+Sij4Jf8AJItD/wC3j/0fJRQB5B+0d/yU%0ALT/+wVH/AOjZa8gr1/8AaO/5KFp//YKj/wDRsteQUxhRRRQAUUUUAFTW8zQT%0AJKpwVOahpRSaurCaTVmeh2twtzaxyqchlBPse4qeuO0TVmsi0ZBZMFsZ6EAn%0Ain3niS5m+WJVjX8z+ZryZ4KTnaOx4U8um6jUdjrWYKpZiAB3JxWdc67ZW2QZ%0AN7j+FOf1rjZru4mOZJnb6k1B171tTwCi/edzpp5XFP33c6KTxVKz/u7dFTtu%0AJJq3b+KIG4miZD6g5zXJUtbywlJq1jplgaMlax2V9rlvHZeZbSB5W+6MYK+5%0ArjndpHLuSWJySaT8aSro0Y0laJpQw8KKtETNGaKK2NwooooA+v8A4Jf8ki0P%0A/t4/9HyUUfBL/kkWh/8Abx/6PkooA7+iiigAooooAKKKKACiiigAooooAKKK%0AKAFoNFFIBKKKKYBRRRQAUUUUAFFFFAH/2Q==)

于是，为了解决这个问题，FCN将不同全局步长下的层之间进行连接。具体网络结构如下图所示

![images](...)

这样一来，随着细节信息的逐渐加入，分割的结果也越来越好。比如下图从左到右分别对应了全局步长32、全局步长16和全局步长8下的结果。最右侧是真值。

![images](...%0AJ/Or1FAoSS2GkkrI/9k=)

# 3.总结与思考
尽管FCN意义重大，在当时来讲效果也相当惊人，但是FCN本身仍然有许多局限。比如：
- 没有考虑全局信息
- 无法解决实例分割问题
- 速度远不能达到实时
- 不能够应对诸如3D点云等不定型数据

基于此，各路研究大神们提出了针对不同局限的各种方法。下图给出了部分研究成果与FCN的关系。

![images](...)